In [1]:
# import numpy as np
import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
from collections import Counter
import itertools

In [2]:
data = pd.read_csv('movie_bd_v5.csv', parse_dates=['release_date'])

In [3]:
data.sample()

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
77,tt1754656,64000000,97571250,The Little Prince,Jeff Bridges|Rachel McAdams|Paul Rudd|Marion C...,Mark Osborne,Growing up isn't the problem... forgetting is.,Based on the best-seller book 'The Little Prin...,92,Adventure|Animation|Fantasy,Onyx Films|Orange Studios|CityMation|On Entert...,2015-07-29,7.5,2015


Видно, что в столбцах cast, director, genres, production_company может быть больше одного наименования

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   imdb_id               1889 non-null   object        
 1   budget                1889 non-null   int64         
 2   revenue               1889 non-null   int64         
 3   original_title        1889 non-null   object        
 4   cast                  1889 non-null   object        
 5   director              1889 non-null   object        
 6   tagline               1889 non-null   object        
 7   overview              1889 non-null   object        
 8   runtime               1889 non-null   int64         
 9   genres                1889 non-null   object        
 10  production_companies  1889 non-null   object        
 11  release_date          1889 non-null   datetime64[ns]
 12  vote_average          1889 non-null   float64       
 13  release_year      

Видно, что пропусков в данных нет

# Предобработка

In [5]:
answers = {} # создадим словарь для ответов

In [6]:
# столбец - прибыль фильма

data.insert(3, 'profit', data['revenue'] - data['budget'])

In [7]:
# столбец - месяц выпуска фильма

data['release_month'] = data['release_date'].map(lambda x : x.month)

In [8]:
# функция создает таблицу истинности по столбцу с упакованными данными
def create_truth_table_by_column(packed_series):
    # распакуем строки в списки
    unpacked_series = packed_series.str.split(pat='|')
    # собираем все уникальные элементы
    all_items = []
    for this_record in unpacked_series:
        for this_item in this_record:
            if not this_item in all_items:
                all_items.append(this_item)
    all_items.sort()    # не обязательно, но понятнее
    # создаем пустую таблицу
    truth_table = pd.DataFrame(data=False, index=range(unpacked_series.shape[0]), columns=all_items)
    # заполняем совпадения
    for i_record in range(unpacked_series.shape[0]):
        for this_item in unpacked_series[i_record]:
            truth_table.iloc[i_record][this_item] = True
    return truth_table

In [9]:
# вынесем жанры в отдельную таблицу
genres = create_truth_table_by_column(data['genres'])
# удалим исходный стоблец
data.drop(columns='genres', inplace=True)

In [10]:
# вынесем режиссеров в отдельную таблицу
directors = create_truth_table_by_column(data['director'])
# удалим исходный стоблец
data.drop(columns='director', inplace=True)

In [11]:
# вынесем актеров в отдельную таблицу
actors = create_truth_table_by_column(data['cast'])
# удалим исходный стоблец
data.drop(columns='cast', inplace=True)

In [12]:
# вынесем киностудии в отдельную таблицу
film_studios = create_truth_table_by_column(data['production_companies'])
# удалим исходный стоблец
data.drop(columns='production_companies', inplace=True)

In [13]:
# что осталось в основной таблице
data.sample()

,imdb_id,budget,revenue,profit,original_title,tagline,overview,runtime,release_date,vote_average,release_year,release_month
1088,tt0315327,80000000,484572835,404572835,Bruce Almighty,In Bruce we trust,"Bruce Nolan toils as a ""human interest"" televi...",101,2003-05-23,6.3,2003,5


# 1. У какого фильма из списка самый большой бюджет?

In [14]:
data[['budget', 'original_title', 'imdb_id']].sort_values(by='budget', ascending = False).head(1)

,budget,original_title,imdb_id
723,380000000,Pirates of the Caribbean: On Stranger Tides,tt1298650


In [15]:
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'

# 2. Какой из фильмов самый длительный (в минутах)?

In [16]:
data[['runtime', 'original_title', 'imdb_id']].sort_values(by='runtime', ascending = False).head(1)

,runtime,original_title,imdb_id
1157,214,Gods and Generals,tt0279111


In [17]:
answers['2'] = '2. Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?





In [18]:
data[['runtime', 'original_title', 'imdb_id']].sort_values(by='runtime', ascending = True).head(1)

,runtime,original_title,imdb_id
768,63,Winnie the Pooh,tt1449283


In [19]:
answers['3'] = '3. Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [20]:
round(data['runtime'].mean())

110

In [21]:
answers['4'] = '2. 110'

# 5. Каково медианное значение длительности фильмов? 

In [22]:
round(data['runtime'].median())

107

In [23]:
answers['5'] = '5. 107'

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [24]:
data[['profit', 'original_title', 'imdb_id']].sort_values(by='profit', ascending = False).head(1)

,profit,original_title,imdb_id
239,2544505847,Avatar,tt0499549


In [25]:
answers['6'] = '5. Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [26]:
data[['profit', 'original_title', 'imdb_id']].sort_values(by='profit', ascending = True).head(1)

,profit,original_title,imdb_id
1245,-165710090,The Lone Ranger,tt1210819


In [27]:
answers['7'] = '5. The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [28]:
data[data['profit'] > 0].shape[0]

1478

In [29]:
answers['8'] = '1. 1478'

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [30]:
data_2008 = data[data['release_year'] == 2008]
data_2008[['profit', 'original_title', 'imdb_id']].sort_values(by='profit', ascending = False).head(1)

,profit,original_title,imdb_id
599,816921825,The Dark Knight,tt0468569


In [31]:
answers['9'] = '4. The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [32]:
data_2012_2014 = data.query('release_year >= 2012 & release_year <= 2014')
data_2012_2014[['profit', 'original_title', 'imdb_id']].sort_values(by='profit', ascending = True).head(1)

,profit,original_title,imdb_id
1245,-165710090,The Lone Ranger,tt1210819


In [33]:
answers['10'] = '5. The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [34]:
genres.sum().sort_values(ascending=False).head(1)

Drama    782
dtype: int64

In [35]:
answers['11'] = '3. Drama'

In [36]:
# альтернатива по комментариям ментора:
# Для решения 11 вопроса также подойдёт функция Counter()
# возможно, что имелось в виду value_counts(),
# но тогда исходные данные надо формировать иначе

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [37]:
genres[data['profit'] > 0].sum().sort_values(ascending=False).head(1)

Drama    560
dtype: int64

In [38]:
answers['12'] = '1. Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [39]:
directors_pivot = pd.DataFrame(data=0, index=directors.columns, columns=['total_revenue'])
for this_director in directors_pivot.index:
    directors_pivot['total_revenue'][this_director] = data[directors[this_director]]['revenue'].sum()
directors_pivot.sort_values(by='total_revenue', ascending=False).head(1)

,total_revenue
Peter Jackson,6490593685


In [40]:
answers['13'] = '5. Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [41]:
# дополняем предыдущий датафрейм
directors_pivot['action_count'] = 0
for this_director in directors_pivot.index:
    directors_pivot['action_count'][this_director] = genres[directors[this_director]]['Action'].sum()
directors_pivot.sort_values(by='action_count', ascending=False).head(1)

,total_revenue,action_count
Robert Rodriguez,930768969,9


In [42]:
answers['14'] = '3. Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [43]:
actors_pivot = pd.DataFrame(data=0, index=actors.columns, columns=['total_revenue_2012'])
for this_actor in actors_pivot.index:
    actor_2012_mask = (actors[this_actor]) & (data['release_year'] == 2012)
    actors_pivot['total_revenue_2012'][this_actor] = data[actor_2012_mask]['revenue'].sum()
actors_pivot.sort_values(by='total_revenue_2012', ascending=False).head(1)

,total_revenue_2012
Chris Hemsworth,2027450773


In [44]:
answers['15'] = '3. Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [45]:
# высокобюджетный == бюджет выше среднего
mean_budget = data['budget'].mean()
# дополняем предыдущий датафрейм
actors_pivot['big_budget_count'] = 0
for this_actor in actors_pivot.index:
    actor_budget_mask = (actors[this_actor]) & (data['budget'] > mean_budget)
    actors_pivot['big_budget_count'][this_actor] = data[actor_budget_mask]['imdb_id'].count()
actors_pivot.sort_values(by='big_budget_count', ascending=False).head(1)

,total_revenue_2012,big_budget_count
Matt Damon,8138788,18


In [46]:
answers['16'] = '3. Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [47]:
genres[actors['Nicolas Cage']].sum().sort_values(ascending=False).head(1)

Action    17
dtype: int64

In [48]:
answers['17'] = '2. Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [49]:
data[film_studios['Paramount Pictures']][['profit', 'original_title', 'imdb_id']].sort_values(by='profit').head(1)

,profit,original_title,imdb_id
925,-64831034,K-19: The Widowmaker,tt0267626


In [50]:
answers['18'] = '1. K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [51]:
years_pivot = pd.pivot_table(data, index='release_year', values='revenue', aggfunc='sum')
years_pivot.sort_values(by='revenue', ascending=False).head(1)

,revenue
release_year,
2015,25449202382


In [52]:
answers['19'] = '5. 2015'

# 20. Какой самый прибыльный год для студии Warner Bros?

In [53]:
# если название студии содержит Warner Bros
wb_studios = list(filter(lambda x : 'Warner Bros' in x, film_studios.columns))
data_wb = data[film_studios[wb_studios].max(axis=1)]
pd.pivot_table(data_wb, index='release_year', values='profit', aggfunc='sum').sort_values(by='profit', ascending=False).head(1)

,profit
release_year,
2014,2295464519


In [54]:
answers['20'] = '1. 2014'

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [55]:
release_months_pivot = pd.pivot_table(data, index='release_month', values='imdb_id', aggfunc='count')
release_months_pivot.sort_values(by='imdb_id', ascending=False).head(1)

,imdb_id
release_month,
9,227


In [56]:
answers['21'] = '4. Сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [57]:
summer_months = [6, 7, 8]
# используем предыдущий датафрейм
release_months_pivot.loc[summer_months].sum()

imdb_id    450
dtype: int64

In [58]:
answers['22'] = '2. 450'

# 23. Какой режиссер выпускает (суммарно по годам) больше всего фильмов зимой? 

In [59]:
winter_months = [12, 1, 2]
winter_movies_index = data[data['release_month'].isin(winter_months)].index
directors.loc[winter_movies_index].sum().sort_values(ascending=False).head(1)

Peter Jackson    7
dtype: int64

In [60]:
answers['23'] = '5. Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [61]:
title_length = data['original_title'].map(lambda x : len(x)).sort_values(ascending=False)
target_movie = film_studios.loc[title_length.index[0]]
target_movie[target_movie]    # используем серию как маску для самой себя

Four By Two Productions                   True
Twentieth Century Fox Film Corporation    True
Name: 1448, dtype: bool

In [62]:
answers['24'] = '5. Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [63]:
mean_words_count = pd.Series(data=0, index=film_studios.columns, name='mean_words_count')
for this_studio in mean_words_count.index:
    mean_words_count[this_studio] = data[film_studios[this_studio]]['overview'].map(lambda x : len(x.split())).mean()
mean_words_count.sort_values(ascending=False).head(1)

Midnight Picture Show    175
Name: mean_words_count, dtype: int64

In [64]:
answers['25'] = '3. Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 

In [65]:
movies_rating = data[['original_title', 'vote_average']]
limit_value = movies_rating.quantile(q=0.99).iloc[0]
movies_rating[movies_rating['vote_average'] > limit_value]

,original_title,vote_average
9,Inside Out,8.0
34,Room,8.0
118,Interstellar,8.0
119,Guardians of the Galaxy,7.9
125,The Imitation Game,8.0
128,Gone Girl,7.9
138,The Grand Budapest Hotel,7.9
370,Inception,7.9
599,The Dark Knight,8.1
872,The Pianist,7.9


In [66]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [67]:
# возможное количество пар актеров - оооооочень много
(len(actors.columns) * (len(actors.columns) - 1)) // 2

5798715

Вариант 1. Перебор всех возможных пар актеров и подсчёт количества фильмов. Выполняется почти 20 мин.

In [68]:
%%script false

most_frequent_pair = {'name': 'nothing & nothing', 'count': 0}
for i_actor_1 in range(len(actors.columns)-1):
    actor_1 = actors.columns[i_actor_1]
    actor_1_column = actors[actor_1]
    for i_actor_2 in range(i_actor_1+1, len(actors.columns)):
        actor_2 = actors.columns[i_actor_2]
        pair_count = (actor_1_column & actors[actor_2]).sum()
        if pair_count > most_frequent_pair['count']:
            pair_name = '{0} & {1}'.format(actor_1, actor_2)
            print('found new pair - ' + pair_name)
            most_frequent_pair['name'] = pair_name
            most_frequent_pair['count'] = pair_count
        elif (pair_count == most_frequent_pair['count']) and (pair_count > 2):
            pair_name = '{0} & {1}'.format(actor_1, actor_2)
            print('same count - ' + pair_name)
print(most_frequent_pair)

Couldn't find program: 'false'


Вариант 2. Проходим по всем фильмам, комбинируем фактических актеров по парам и сразу считаем.

In [69]:
# от этой функции можно избавиться, т.к. она используется только в одном месте
def make_pair_name(name_1, name_2):
    return '{0} & {1}'.format(name_1, name_2)

actors_pair = pd.Series(name='movies_count', dtype='int64')
for this_movie in actors.index:
    this_movie_cast = actors.loc[this_movie][actors.loc[this_movie]].index    # задействованные актеры
    for i_actor_1 in range(this_movie_cast.shape[0]-1):
        actor_1 = this_movie_cast[i_actor_1]
        for i_actor_2 in range(i_actor_1+1, this_movie_cast.shape[0]):
            actor_2 = this_movie_cast[i_actor_2]
            pair_name = make_pair_name(actor_1, actor_2)
            if pair_name in actors_pair.index:
                actors_pair[pair_name] = actors_pair[pair_name] + 1
            else:
                actors_pair[pair_name] = 1

actors_pair.sort_values(ascending=False).head(3)

Daniel Radcliffe & Emma Watson     8
Emma Watson & Rupert Grint         8
Daniel Radcliffe & Rupert Grint    8
Name: movies_count, dtype: int64

In [70]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint'

In [71]:
# В 27-м для лаконичности можно также использовать combinations (из библиотеки itertools)
actors_pair = pd.Series(name='movies_count', dtype='int64')
for this_movie in actors.index:
    this_movie_cast = actors.loc[this_movie][actors.loc[this_movie]].index    # задействованные актеры
    for this_pair in itertools.combinations(this_movie_cast, 2):
        pair_name = str(this_pair)
        if pair_name in actors_pair.index:
            actors_pair[pair_name] = actors_pair[pair_name] + 1
        else:
            actors_pair[pair_name] = 1

actors_pair.sort_values(ascending=False).head(3)
# да, так немного понятнее, и будет проще сделать, например, тройки актеров

('Daniel Radcliffe', 'Emma Watson')     8
('Emma Watson', 'Rupert Grint')         8
('Daniel Radcliffe', 'Rupert Grint')    8
Name: movies_count, dtype: int64

In [72]:
# тройки актеров
actors_pair = pd.Series(name='movies_count', dtype='int64')
for this_movie in actors.index:
    this_movie_cast = actors.loc[this_movie][actors.loc[this_movie]].index    # задействованные актеры
    for this_pair in itertools.combinations(this_movie_cast, 3):
        pair_name = str(this_pair)
        if pair_name in actors_pair.index:
            actors_pair[pair_name] = actors_pair[pair_name] + 1
        else:
            actors_pair[pair_name] = 1

actors_pair.sort_values(ascending=False).head(6)
# Поттер, Сумерки, Форсаж, Люди Хэ, Пила, Голодные игры

('Daniel Radcliffe', 'Emma Watson', 'Rupert Grint')          8
('Kristen Stewart', 'Robert Pattinson', 'Taylor Lautner')    5
('Jordana Brewster', 'Paul Walker', 'Vin Diesel')            4
('Hugh Jackman', 'Ian McKellen', 'Patrick Stewart')          4
('Betsy Russell', 'Costas Mandylor', 'Tobin Bell')           4
('Josh Hutcherson', 'Liam Hemsworth', 'Woody Harrelson')     4
Name: movies_count, dtype: int64

In [73]:
data = pd.read_csv('movie_bd_v5.csv', parse_dates=['release_date'])
# вариант 27 из слака - очень здорово - быстрее в 20 раз
cast_pairs = data["cast"].apply(lambda s: str(s).split('|')).apply(
    lambda i: list(itertools.combinations(sorted(i), 2))).values.sum()
Counter(cast_pairs).most_common(3)

[(('Daniel Radcliffe', 'Emma Watson'), 8),
 (('Daniel Radcliffe', 'Rupert Grint'), 8),
 (('Emma Watson', 'Rupert Grint'), 8)]


# Submission

In [74]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '5. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe & Rupert Grint'}

In [75]:
# и убедиться что ни чего не пропустил)
len(answers)

27